In [1]:
# %% [markdown]
# # 🏠 Train House Price Prediction Model
# Uses California Housing dataset (built into sklearn)

# %%
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import joblib
import os

# Ensure models/ folder exists
os.makedirs("models", exist_ok=True)

# %% [markdown]
# ## Load & Prepare Data
# %%
data = fetch_california_housing(as_frame=True)
df = data.frame

X = df.drop("MedHouseVal", axis=1)
y = df["MedHouseVal"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# %% [markdown]
# ## Train & Tune Random Forest (Best Performer)
# %%
print("Training Random Forest with hyperparameter tuning...")

rf = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [100],
    'max_depth': [20],
    'min_samples_split': [5],
    'min_samples_leaf': [2]
}

grid_search = GridSearchCV(
    rf, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1
)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_
print("✅ Best params:", grid_search.best_params_)

# Evaluate
y_pred = best_rf.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"🎯 Test RMSE: {rmse:.4f} | R²: {r2:.4f}")

# %% [markdown]
# ## Save Model & Scaler (For App)
# %%
# Save model
joblib.dump(best_rf, "models/best_model.pkl")
print("✅ Model saved to models/best_model.pkl")

# Save feature names for app validation
joblib.dump(list(X.columns), "models/feature_names.pkl")

# Optional: Save scaler if using linear models
scaler = StandardScaler()
scaler.fit(X_train)
joblib.dump(scaler, "models/scaler.pkl")
print("✅ Scaler saved")

print("\n🎉 Training complete! Now run: streamlit run app.py")

Training Random Forest with hyperparameter tuning...
✅ Best params: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
🎯 Test RMSE: 0.5044 | R²: 0.8059
✅ Model saved to models/best_model.pkl
✅ Scaler saved

🎉 Training complete! Now run: streamlit run app.py


In [2]:
# Save test score for app display
with open("models/test_r2.txt", "w") as f:
    f.write(f"{r2:.4f}")
print("✅ Test R² saved to models/test_r2.txt")

✅ Test R² saved to models/test_r2.txt


In [3]:
import os
print("\n📁 Files in models/:")
for f in os.listdir("models"):
    size = os.path.getsize(f"models/{f}") / (1024*1024)
    print(f"  - {f}: {size:.1f} MB")


📁 Files in models/:
  - best_model.pkl: 47.6 MB
  - feature_names.pkl: 0.0 MB
  - scaler.pkl: 0.0 MB
  - test_r2.txt: 0.0 MB
